In [1]:
import geopandas as gpd
import pandas as pd
import pandas
import math
from shapely.geometry import Point, LineString, Polygon, MultiPoint, MultiLineString, MultiPolygon
from geographiclib.geodesic import Geodesic
from shapely.ops import split
from shapely import ops
from pyproj import Proj, Transformer

In [2]:
buildings= gpd.read_file('../tests/data/buildings.geojson')
road= gpd.read_file('../tests/data/road.geojson')

In [3]:
# for d in buildings.columns:
#     print(d)
road['end'] = None
road['start'] = None
road['start_point_x'] = None
road['start_point_y'] = None
road['end_point_x'] = None
road['end_point_y'] = None
road['length'] = None
road['bearing'] = None
road['intersected'] = None
road['mother_road']='None'

buildings['road_direction'] = 'None'
buildings['distance2_intersecting_road'] = 0
buildings['road_length_d_code'] = ''

# print(buildings)
# print(road)

In [4]:
class bdcPoint:
    	
	def __init__(self):
		
		self.x = 0
		self.y = 0

# Constant integers for directions
RIGHT = 1
LEFT = -1
ZERO = 0

def directionOfPoint(A, B, P):
    """[Provides direction of the point in respect to line]

    Args:
        A ([type]): [Starting Point of Line]
        B ([type]): [Ending Point of Line]
        P ([type]): [Point of Interest]

    Returns:
        [Direction(int)]: [Left,Right and Zero : -1,1,0 ( Left direction, Right Direction , On Line)]
    """
    global RIGHT, LEFT, ZERO
	
	# Subtracting co-ordinates of
	# point A from B and P, to
	# make A as origin
 
    B.x -= A.x
    B.y -= A.y
    P.x -= A.x
    P.y -= A.y

    # Determining cross Product
    """"The Cross-Product has an interesting Property which will be used to determine direction of a point from a line segment. That is, the cross-product of two points is positive if and only if the angle of those point at origin (0, 0) is in counter-clockwise. And conversely the cross-product is negative if and only if the angle of those point at origin is in clockwise direction."""
    cross_product = B.x * P.y - B.y * P.x
    # Return RIGHT if cross product is positive
    if (cross_product > 0):
        return RIGHT
        
    # Return LEFT if cross product is negative
    if (cross_product < 0):
        return LEFT

    # Return ZERO if cross product is zero
    return ZERO

def addDirection_BuildingCode(direction,building_length,road_length,index,road_index):
    #right-even, left-odd number 
    # print(length)
    building_length=int(building_length)
    if road_length < 60:
        associated_road=road.iloc[road_index]

        if associated_road.mother_road == 'start':
            mother_road_code=associated_road.start
        else :
            mother_road_code=associated_road.end

        mother_road= road.loc[road['NEW'] == mother_road_code]

        mr_geom= mother_road.geometry
        d = {'col1': ['name1'], 'geometry': [Point(associated_road.start_point_x, associated_road.start_point_y)]}
        motherroad_start_point = {'col1': ['name1'], 'geometry': [Point(mother_road.start_point_x, mother_road.start_point_y)]}
        
        gdf = gpd.GeoDataFrame(d, crs=4326)
        mother_df = gpd.GeoDataFrame(motherroad_start_point, crs=4326)
        
        sp_point=gdf.iloc[0]
        mother_st_p=mother_df.iloc[0]
        for line in mr_geom:
            # print(line)
            for l in line:
                mother_line=l
                break

        result=split(mother_line,sp_point.geometry)
        for r  in result:
            splitted_line=r
            splitted_line.srid = 4326
            if splitted_line.contains(mother_st_p.geometry):
                break
            else:
                continue
        my_transformer = Transformer.from_crs('EPSG:4326', 'EPSG:32644', always_xy=True)
        geom_transformed = ops.transform(my_transformer.transform, splitted_line)
        
        buildings.at[index,'road_length_d_code']=str(int(geom_transformed.length))+"/"
        if (direction == 1):
            # print("Right Direction")
            buildings.at[index,'road_direction']="right"
            buildings.at[index,'road_length_d_code']+=str(int(round_up_to_even(building_length)))

        elif (direction == -1):
            # print("Left Direction")
            buildings.at[index,'road_direction']="left"
            buildings.at[index,'road_length_d_code']+=str(int(round_up_to_odd(building_length)))
        else:
            print("On  Line")
            buildings.at[index,'road_direction']="on_line"
    else :
        if (direction == 1):
            # print("Right Direction")
            buildings.at[index,'road_direction']="right"
            buildings.at[index,'road_length_d_code']=str(int(round_up_to_even(building_length)))

        elif (direction == -1):
            # print("Left Direction")
            buildings.at[index,'road_direction']="left"
            buildings.at[index,'road_length_d_code']=str(int(round_up_to_odd(building_length)))
        else:
            print("On  Line")
            buildings.at[index,'road_direction']="on_line"
    
    
#     print(buildings.at[index,'road_length_d_code'])
            
def getStartEndPoint(line,type):
    if len(line)==1 or type=="road" :
       
        df_line = line.geometry.boundary
        # print(len(df_line))
        # print(line_bound)
        # df_line=line_bound.explode(index_parts=True)
        start_point,end_point=df_line[0],df_line[1]

        return start_point,end_point
    else:
        raise ValueError ("Line contains more than one code")

def getLineLength(line):
    projected_line = line.to_crs(epsg=32644)
    length= projected_line.geometry.length
    float_length=length.tolist()
    return float_length

def round_up_to_even(f):
    return math.ceil(f / 2.) * 2

def round_up_to_odd(f):
    return math.ceil(f) // 2 * 2 + 1

def addStartEndPoint(start_point,end_point,index):
    
    road.at[index,'start_point_x']=float(start_point.x)
    road.at[index,'start_point_y']=float(start_point.y)
    
    road.at[index,'end_point_x']=float(end_point.x)
    road.at[index,'end_point_y']=float(end_point.y)

def ReviseStartEndDateWithBearing(bearing,index):
    # default direction is west to east , south to north 
    # print("ma revise garna aako dubai vettera")
    if bearing is None :
        bearing=0
    bearing=int(bearing)
    def assign_start():
        print("start")
        
        road.at[index,'mother_road']='start'

    def assign_end():
    # east_west/west_east direction , assign to east
        print("end")
        road.at[index,'mother_road']='end'

    
    if ((bearing >= 0) and (bearing<=45)) :
        #    North_South/south_north Direction, assign to south
        
        assign_start()
        
    if ((bearing > 45) and (bearing<135)) :
        
        # east_west/west_east direction , assign to east
        assign_end()
    
    if ((bearing >= 135) and (bearing<=225)) :
        #    North_South/south_north Direction, assign to south
        
        assign_start()
    if ((bearing > 225) and (bearing<315)) :
            
        # east_west/west_east direction , assign to east
        assign_end()
    if ((bearing >= 315) and (bearing<=360)) :
    #    North_South/south_north Direction, assign to south
        
        assign_start()      

def FindIntersectedFeatures(start_point,end_point,line_row,index_road):
    for index, row in road.iterrows():
        line=row.geometry        
        if str(row.NEW) != str(line_row.NEW):
            if line.distance(start_point) < 1e-8 :
                # print("start found "+ str(row.NEW))
                # print("mero start vetiyo guys ")
                road.at[index_road,'start']=str(row.NEW)
    
            if line.distance(end_point) < 1e-8:
                # print("end found "+ str(row.NEW))
                road.at[index_road,'end']=str(row.NEW)
                # print("mero end vetiyo guys ")


def scan_I_feature(line_row,index_road):
    
        if ((line_row.start!= None) and (line_row.end== None)) :
            road.at[index_road,'mother_road']='start'
        if ((line_row.start== None) and (line_row.end != None)) :
            road.at[index_road,'mother_road']='end'
    
        if ((line_row.start!= None) and (line_row.end != None)) :
        #both start and end exist which means line is in between roads 
            road.at[index_road,'mother_road']='None'
            ReviseStartEndDateWithBearing(line_row.bearing,index_road)

                    
def CalculateBearing(start_point,end_point,index):
    brng = Geodesic.WGS84.Inverse(start_point.y, start_point.x, end_point.y, end_point.x)['azi1']
    road.at[index,'bearing']=brng

def is_number(string):
    try:
        float(string)
        return True
    except ValueError:
        return False    
    
def AssignIntersectedPoint(start_point_assumed,end_point_assumed,row,index):
#     print(row)
    if  row.mother_road :
        mother_road_direction = row.mother_road
        mother_road_code=None
        if mother_road_direction == 'start':
            mother_road_code=row.start
        elif mother_road_direction == 'end' :
            mother_road_code=row.end
        mother_road= road.loc[road['NEW'] == mother_road_code].geometry
        current_road = row.geometry
        mother_road_geom=mother_road
        for l in mother_road:
            for single in l:
                mother_road_geom=single
            break
#         print(mother_road_geom)
        for l in current_road:
            road_geom=l
            break
#         print(road_geom)
        start_point=mother_road_geom.intersection(road_geom)
        print(start_point)
        st=str(type(start_point))
#         print(st)
#         geoseries=f"""<class 'geopandas.geoseries.GeoSeries'>"""
#         print(geoseries)
    
        if isinstance(start_point,pandas.core.series.Series) :
            print(type(start_point))
            print('not counted')
            
            road.at[index,'intersected']='No'
        else:
            print(start_point)
            try:
                x=start_point.x
                y=start_point.y
                road.at[index,'start_point_x']=float(x)
                road.at[index,'start_point_y']=float(y)              
            except:
                try:
                    for p in start_point:
                        x=p.x
                        y=p.y
                        break
                    road.at[index,'start_point_x']=x
                    road.at[index,'start_point_y']=y
                except:
                    try:
                        df_line = start_point.boundary
                        start=df_line[0]
                        road.at[index,'start_point_x']=start.x
                        road.at[index,'start_point_y']=start.y
                    except:
                        print('i didnt assigned')
                        pass
#                     road.at[index,'start_point_x']=0
#                     road.at[index,'start_point_y']=0
 

        
def get_building_projected_road_length(road_linked_wrt_point,row):
    road_code = row.NEW
#     print(road_linked_wrt_point)
    linked_road_geometry = road_linked_wrt_point.geometry
    building_geometry = row.geometry
    degree_length_for_building=linked_road_geometry.project(building_geometry)
    # 5m=0.00005 in degree 
    building_point_projected_on_road=linked_road_geometry.interpolate(degree_length_for_building) #end_always 
    for r in building_point_projected_on_road:
        building_point_projected_on_road_single_geom=r
        break
    try:
        projected_building_on_road = {'col1': ['name1'], 'geometry': [Point(building_point_projected_on_road_single_geom.x, building_point_projected_on_road_single_geom.y)]}
    except:
        print(projected_building_on_road)
        raise("found error")

    road_link_wrt_point_x=road_linked_wrt_point.start_point_x
    road_link_wrt_point_y=road_linked_wrt_point.start_point_y
    try:
        road_start_point = {'col1': ['name1'], 'geometry': [Point(road_link_wrt_point_x,road_link_wrt_point_y)]}

        projected_building_on_road_gdf = gpd.GeoDataFrame(projected_building_on_road, crs=4326)


        road_start_df = gpd.GeoDataFrame(road_start_point, crs=4326)


        splitting_point=projected_building_on_road_gdf.iloc[0]
        start_point=road_start_df.iloc[0]
#         print(start_point.geometry)
        for line in linked_road_geometry:
            # print(line)
            for l in line:
                mother_line=l
                break
        result=split(mother_line,splitting_point.geometry.buffer(0.0000001))

        for r  in result:
                splitted_line=r
                splitted_line.srid = 4326
                if splitted_line.buffer(0.00001).contains(start_point.geometry.buffer(0.0000001)) == True:

                    break
                else:
                    continue


        
        my_transformer = Transformer.from_crs('EPSG:4326', 'EPSG:32644', always_xy=True)
        geom_transformed = ops.transform(my_transformer.transform, splitted_line)

        clipped_length=geom_transformed.length
        return clipped_length ,splitted_line ,building_point_projected_on_road_single_geom,start_point.geometry
    except:
        pass



    
                   


In [5]:
def CalculateTouchLine(index,row):
    
    # print(row)
    start_point,end_point=getStartEndPoint(row,"road")
    addStartEndPoint(start_point,end_point,index)
    CalculateBearing(start_point,end_point,index)
    FindIntersectedFeatures(start_point,end_point,row,index)
    scan_I_feature(row,index)
    AssignIntersectedPoint(start_point,end_point,row,index)
    # break


In [12]:
for index,row in road.iterrows():
    CalculateTouchLine(index,row)
    print(index)
print("Sucessfully added : startpoint, endpoint , Start touching Line , End Touching Line , bearing ")   

GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
0
end
POINT (83.47182876258739 27.59323857200252)
POINT (83.47182876258739 27.59323857200252)
1
POINT (83.47268607010614 27.60246986818993)
POINT (83.47268607010614 27.60246986818993)
2
POINT (83.47040628242405 27.57787412609008)
POINT (83.47040628242405 27.57787412609008)
3
POINT (83.47172783257349 27.59225408505802)
POINT (83.47172783257349 27.59225408505802)
4
end
POINT (83.46736454413843 27.56698659255727)
POINT (83.46736454413843 27.56698659255727)
5
POINT (83.47153400529015 27.59036561757927)
POINT (83.47153400529015 27.59036561757927)
6
end
POINT (83.47187124206425 27.59364565192533)
POINT (83.47187124206425 27.59364565192533)
7
end
POINT (83.47263962641767 27.60196558594311)
POINT (83.47263962641767 27.60196558594311)
8
POINT (83.47213499163661 27.59644264511201)
POINT (83.47213499163661 27.59644264511201)
9
POINT (83.47071132616685 27.58135272534804)
POINT (83.47071132616685 27.58135272534804)
1

start
POINT (83.45892516306589 27.56683897154136)
POINT (83.45892516306589 27.56683897154136)
89
POINT (83.47150689522709 27.59260663009195)
POINT (83.47150689522709 27.59260663009195)
90
POINT (83.4702157331696 27.59357258021043)
POINT (83.4702157331696 27.59357258021043)
91
POINT (83.46215994622457 27.57253243552356)
POINT (83.46215994622457 27.57253243552356)
92
start
POINT (83.46231292090465 27.56762419390145)
POINT (83.46231292090465 27.56762419390145)
93
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
94
POINT (83.46899543768933 27.59266431350733)
POINT (83.46899543768933 27.59266431350733)
95
POINT (83.46991239903804 27.58034128302387)
POINT (83.46991239903804 27.58034128302387)
96
end
POINT (83.46804093504119 27.57045325151637)
POINT (83.46804093504119 27.57045325151637)
97
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
98
start
POINT (83.46468126302995 27.57313549121)
POINT (83.46468126302995 27.57313549121

start
POINT (83.43251684727471 27.60744073140467)
POINT (83.43251684727471 27.60744073140467)
180
POINT (83.43736914746825 27.5958884028276)
POINT (83.43736914746825 27.5958884028276)
181
end
POINT (83.4548427625835 27.59128840923273)
POINT (83.4548427625835 27.59128840923273)
182
POINT (83.43558870385567 27.59570894581123)
POINT (83.43558870385567 27.59570894581123)
183
end
POINT (83.43478050381725 27.59557949020137)
POINT (83.43478050381725 27.59557949020137)
184
start
POINT (83.45268863357364 27.59713899556823)
POINT (83.45268863357364 27.59713899556823)
185
end
POINT (83.45336136153765 27.58646731821221)
POINT (83.45336136153765 27.58646731821221)
186
POINT (83.44703032768751 27.59477817627186)
POINT (83.44703032768751 27.59477817627186)
187
POINT (83.43686148377054 27.59586403299875)
POINT (83.43686148377054 27.59586403299875)
188
POINT (83.45430597434131 27.58973506691984)
POINT (83.45430597434131 27.58973506691984)
189
start
POINT (83.45245644840645 27.58064707859984)
POINT (83.

end
POINT (83.46570238295868 27.56816980449292)
POINT (83.46570238295868 27.56816980449292)
268
end
POINT (83.46301775466571 27.59483154743799)
POINT (83.46301775466571 27.59483154743799)
269
end
MULTIPOINT (83.46700765987566 27.58456492653016, 83.46824549393443 27.58456295341762)
MULTIPOINT (83.46700765987566 27.58456492653016, 83.46824549393443 27.58456295341762)
270
POINT (83.46524160811379 27.56899020473423)
POINT (83.46524160811379 27.56899020473423)
271
POINT (83.46859447945172 27.57603419052555)
POINT (83.46859447945172 27.57603419052555)
272
POINT (83.46898322399664 27.57755337838574)
POINT (83.46898322399664 27.57755337838574)
273
POINT (83.46855191094198 27.58070076273009)
POINT (83.46855191094198 27.58070076273009)
274
POINT (83.46075547318662 27.56988600503513)
POINT (83.46075547318662 27.56988600503513)
275
POINT (83.4677398267317 27.57107695743571)
POINT (83.4677398267317 27.57107695743571)
276
POINT (83.46969148867134 27.58437466955542)
POINT (83.46969148867134 27.584374

351
POINT (83.43489163484111 27.604195213938)
POINT (83.43489163484111 27.604195213938)
352
POINT (83.45772744956219 27.59826068398343)
POINT (83.45772744956219 27.59826068398343)
353
POINT (83.43470481417592 27.60398744986099)
POINT (83.43470481417592 27.60398744986099)
354
POINT (83.45765913526094 27.59677169985412)
POINT (83.45765913526094 27.59677169985412)
355
POINT (83.43969072124986 27.5970122981791)
POINT (83.43969072124986 27.5970122981791)
356
POINT (83.45024668373753 27.59916563668486)
POINT (83.45024668373753 27.59916563668486)
357
POINT (83.45827922495104 27.60336004024421)
POINT (83.45827922495104 27.60336004024421)
358
POINT (83.45907372027602 27.5720712128163)
POINT (83.45907372027602 27.5720712128163)
359
POINT (83.45985714129409 27.57115195909569)
POINT (83.45985714129409 27.57115195909569)
360
POINT (83.44469313987656 27.59143003986452)
POINT (83.44469313987656 27.59143003986452)
361
POINT (83.44044875170857 27.5985606006912)
POINT (83.44044875170857 27.5985606006912

POINT (83.45649176398376 27.6066034418954)
POINT (83.45649176398376 27.6066034418954)
436
POINT (83.45803121177107 27.60130483425075)
POINT (83.45803121177107 27.60130483425075)
437
POINT (83.46060553551712 27.57641092372518)
POINT (83.46060553551712 27.57641092372518)
438
MULTIPOINT (83.43251684727471 27.60744073140467, 83.43450694264135 27.61119965294171)
MULTIPOINT (83.43251684727471 27.60744073140467, 83.43450694264135 27.61119965294171)
439
POINT (83.43980928067384 27.59757897898675)
POINT (83.43980928067384 27.59757897898675)
440
POINT (83.45838273877246 27.60260481416088)
POINT (83.45838273877246 27.60260481416088)
441
POINT (83.43361060994175 27.60681093347785)
POINT (83.43361060994175 27.60681093347785)
442
POINT (83.45861981445296 27.57372140761692)
POINT (83.45861981445296 27.57372140761692)
443
start
MULTIPOINT (83.43739226903807 27.59472507310369, 83.43741506865058 27.59541615802931)
MULTIPOINT (83.43739226903807 27.59472507310369, 83.43741506865058 27.59541615802931)
444


518
POINT (83.4671872077235 27.60314989954827)
POINT (83.4671872077235 27.60314989954827)
519
POINT (83.46398046902351 27.57362415403151)
POINT (83.46398046902351 27.57362415403151)
520
end
POINT (83.46578593627203 27.59805768361525)
POINT (83.46578593627203 27.59805768361525)
521
POINT (83.46807154166839 27.59511797611378)
POINT (83.46807154166839 27.59511797611378)
522
POINT (83.46831610690288 27.57573069901258)
POINT (83.46831610690288 27.57573069901258)
523
POINT (83.46398046902351 27.57362415403151)
POINT (83.46398046902351 27.57362415403151)
524
POINT (83.46702280445891 27.5701549014276)
POINT (83.46702280445891 27.5701549014276)
525
POINT (83.46470467867806 27.59394376549244)
POINT (83.46470467867806 27.59394376549244)
526
POINT (83.46523466624689 27.5877407798132)
POINT (83.46523466624689 27.5877407798132)
527
POINT (83.46450854643234 27.59885341994942)
POINT (83.46450854643234 27.59885341994942)
528
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not cou

end
POINT (83.44068258173581 27.5996109297007)
POINT (83.44068258173581 27.5996109297007)
605
POINT (83.45800831683039 27.57002911505168)
POINT (83.45800831683039 27.57002911505168)
606
POINT (83.42592397581092 27.59512808449358)
POINT (83.42592397581092 27.59512808449358)
607
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
608
POINT (83.42945518360989 27.59468555599369)
POINT (83.42945518360989 27.59468555599369)
609
start
POINT (83.46172384338075 27.59493282189226)
POINT (83.46172384338075 27.59493282189226)
610
POINT (83.46114291821118 27.5949150404968)
POINT (83.46114291821118 27.5949150404968)
611
POINT (83.46233024274659 27.60370883629383)
POINT (83.46233024274659 27.60370883629383)
612
end
POINT (83.42751669044299 27.60211535364073)
POINT (83.42751669044299 27.60211535364073)
613
end
POINT (83.45540072626255 27.60452921405204)
POINT (83.45540072626255 27.60452921405204)
614
end
POINT (83.46179254978642 27.59531565969002)
POINT (83.4617925497864

POINT (83.45169005335333 27.57981610952589)
POINT (83.45169005335333 27.57981610952589)
694
POINT (83.42459481559831 27.56935712363241)
POINT (83.42459481559831 27.56935712363241)
695
POINT (83.42706207775274 27.59385875797534)
POINT (83.42706207775274 27.59385875797534)
696
end
POINT (83.46222015132285 27.58936818939479)
POINT (83.46222015132285 27.58936818939479)
697
LINESTRING EMPTY
LINESTRING EMPTY
i didnt assigned
698
POINT (83.46244568961691 27.57812763148507)
POINT (83.46244568961691 27.57812763148507)
699
POINT (83.46473460271977 27.58787808200793)
POINT (83.46473460271977 27.58787808200793)
700
start
LINESTRING EMPTY
LINESTRING EMPTY
i didnt assigned
701
POINT (83.45112073303545 27.60051097749965)
POINT (83.45112073303545 27.60051097749965)
702
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
703
POINT (83.45112509744536 27.60052363545748)
POINT (83.45112509744536 27.60052363545748)
704
POINT (83.42440234898925 27.57122346329226)
POINT (83.424

GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
787
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
788
start
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
789
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
790
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
791
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
792
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
793
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
794
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
795
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
796
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
797
start
GeoSeries([], dtype: geometry)


In [75]:
def assign_building_code(index,row):  

    road_linked_wrt_point= road.loc[road['NEW'] == row.NEW]
#     print(road_linked_wrt_point)
    for l in road_linked_wrt_point.geometry:
        line_geom=l
        break

    my_transformer = Transformer.from_crs('EPSG:4326', 'EPSG:32644', always_xy=True)
    geom_transformed = ops.transform(my_transformer.transform, line_geom)

    road_length=geom_transformed.length

    clipped_length,splitted_mother_road_side,building_point_projected_on_road_single_geom,start_point=get_building_projected_road_length(road_linked_wrt_point,row)

    splitted_with_buffer_boi=split(splitted_mother_road_side,building_point_projected_on_road_single_geom.buffer(0.0001)) 

    for r  in splitted_with_buffer_boi:
        roi_line=r
        roi_line.srid = 4326
        if roi_line.buffer(0.00001).contains(start_point.buffer(0.0000001)) == False:
            break
        else:
            continue

    df_line = roi_line.boundary
    start_roi,end_roi=df_line[0],df_line[1]
#     print(start_roi)
#     print(end_roi)
#     print(building_point_projected_on_road_single_geom)
    if (round(start_roi.x,6) == round(building_point_projected_on_road_single_geom.x,4)) and (round(start_roi.y,4) == round(building_point_projected_on_road_single_geom.y,4)) :
        start_roi=end_roi
        end_roi=building_point_projected_on_road_single_geom
        print("start_end changed")

    road_index=road_linked_wrt_point.index[0]
#         road.at[road_index,'length']=length[0]       
    start = bdcPoint()
    end = bdcPoint()
    point = bdcPoint()
    poi = row.geometry

    start.x,start.y,end.x,end.y,point.x,point.y=start_roi.x,start_roi.y,end_roi.x,end_roi.y,poi.x,poi.y

    direction=directionOfPoint(start,end,point)
    # print(direction)
    # break
#     print(clipped_length)
#     print(direction)
    buildings.at[index,'distance2_intersecting_road']=clipped_length
    road.at[road_index,'length']=road_length
    
    
    addDirection_BuildingCode(direction,clipped_length,road_length,index,road_index)
    
        
#     print("added building code")


In [5]:
for index,row in buildings.iterrows():
    try:
        assign_building_code(index,row)
        print(buildings.iloc[index].distance2_intersecting_road)
        print(index)
    except:
        continue

6545


NameError: name 'assign_building_code' is not defined

In [ ]:
for index, row in buildings.iterrows():
    print(row)
    try:
        assign_building_code(index,row)
        print(index)
        print(buildings.iloc[index].road_length_d_code)
    except:
        continue
print("-------Sucess ------ added: road_length_d_code , length of road , direction of building ")
    

OBJECTID_1                                                                   1
NEW                                                                      TS108
geometry                       POINT Z (83.46114063663111 27.56682068562617 0)
road_direction                                                           right
distance2_intersecting_road                                                669
road_length_d_code                                                         670
Name: 0, dtype: object
POINT (83.46121882688092 27.56703169674663)
POINT (83.46112700544703 27.56699244916708)
POINT (83.46112690583823 27.56699244120398)
1
0
670
OBJECTID_1                                                                   2
NEW                                                                      TS108
geometry                       POINT Z (83.46127483796346 27.56691919556448 0)
road_direction                                                           right
distance2_intersecting_road                    

C:\Users\Kshitiz\AppData\Local\Temp/ipykernel_15456/3255070409.py:300: UserWarning: Geometry is in a geographic CRS. Results from 'interpolate' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  building_point_projected_on_road=linked_road_geometry.interpolate(degree_length_for_building) #end_always


POINT (83.46131587809036 27.56707429701394)
POINT (83.46122447656343 27.56703417665375)
POINT (83.46122438507041 27.56703413649323)
1
1
660
OBJECTID_1                                                                   3
NEW                                                                      TS108
geometry                       POINT Z (83.46143473742325 27.56675929610469 0)
road_direction                                                           right
distance2_intersecting_road                                                650
road_length_d_code                                                         650
Name: 2, dtype: object
POINT (83.4613910974706 27.56710731428105)
POINT (83.46129969594367 27.56706719392086)
POINT (83.46129960445064 27.56706715376034)
1
2
650
OBJECTID_1                                                                   4
NEW                                                                      TW974
geometry                       POINT Z (83.45739143496678 27.56961

POINT (83.45931654091838 27.56831833242978)
POINT (83.4592802331512 27.56822534905178)
POINT (83.45931657726248 27.56831842550624)
1
14
170
OBJECTID_1                                                                  16
NEW                                                                      TW799
geometry                       POINT Z (83.46005452809408 27.56814296512579 0)
road_direction                                                           right
distance2_intersecting_road                                               3844
road_length_d_code                                                        3844
Name: 15, dtype: object
POINT (83.45932108323635 27.56833041661878)
POINT (83.45928492997012 27.56823737749976)
POINT (83.45932110798134 27.56833051338576)
1
15
170
OBJECTID_1                                                                 17
NEW                                                                     TW799
geometry                       POINT Z (83.45952343166113 27.5683

POINT (83.45950584879795 27.56905295480194)
POINT (83.45948112854393 27.56895628458229)
POINT (83.45950587354294 27.56905305156892)
1
28
252
OBJECTID_1                                                                 30
NEW                                                                     TW799
geometry                       POINT Z (83.45970427093539 27.5690909422932 0)
road_direction                                                           None
distance2_intersecting_road                                                 0
road_length_d_code                                                           
Name: 29, dtype: object
POINT (83.45952712059953 27.5691361396159)
POINT (83.45950240034551 27.56903946939626)
POINT (83.45952714534452 27.56913623638289)
1
29
262
OBJECTID_1                                                                  31
NEW                                                                      TW799
geometry                       POINT Z (83.45970236707063 27.56917279

...

798
1552/60
OBJECTID_1                                                                 800
NEW                                                                     TW1346
geometry                       POINT Z (83.45870331200831 27.60205959117724 0)
road_direction                                                            None
distance2_intersecting_road                                                  0
road_length_d_code                                                            
Name: 799, dtype: object
POINT (83.45859886541132 27.60205806613965)
POINT (83.458694803393 27.6020304226278)
POINT (83.45869489942702 27.60203039495662)
1
799
1552/46
OBJECTID_1                                                                 801
NEW                                                                     TW1346
geometry                       POINT Z (83.45841682487662 27.60214144477192 0)
road_direction                                                            None
distance2_intersecting_road   

POINT (83.45802034826161 27.60288306455742)
POINT (83.45802343598679 27.60278331087066)
POINT (83.45802347310484 27.60278321807112)
-1
812
81
OBJECTID_1                                                                 814
NEW                                                                     TW1360
geometry                       POINT Z (83.45792094499569 27.60247266598083 0)
road_direction                                                            None
distance2_intersecting_road                                                  0
road_length_d_code                                                            
Name: 813, dtype: object
POINT (83.45802034826161 27.60288306455742)
POINT (83.45802343598679 27.60278331087066)
POINT (83.45802347310484 27.60278321807112)
-1
813
81
OBJECTID_1                                                                 815
NEW                                                                     TW1360
geometry                       POINT Z (83.45780577871375 2

POINT (83.45807492748526 27.60297922665141)
POINT (83.45802102195087 27.60289519287396)
POINT (83.45802101641131 27.6028950931461)
-1
826
69
OBJECTID_1                                                                828
NEW                                                                    TW1360
geometry                       POINT Z (83.45818649321217 27.6031855522715 0)
road_direction                                                           None
distance2_intersecting_road                                                 0
road_length_d_code                                                           
Name: 827, dtype: object
POINT (83.45808962559575 27.60327034678811)
POINT (83.45810644197208 27.60317196981472)
POINT (83.45810645880529 27.60317187133927)
1
827
34
OBJECTID_1                                                                 829
NEW                                                                     TW1360
geometry                       POINT Z (83.45811986783769 27.603386

POINT (83.45655760541091 27.60163932485327)
POINT (83.45646109209527 27.60166467150319)
POINT (83.45646099548534 27.60166469687521)
1
840
12
OBJECTID_1                                                                 842
NEW                                                                     TW1256
geometry                       POINT Z (83.45644567912484 27.60146770866788 0)
road_direction                                                            None
distance2_intersecting_road                                                  0
road_length_d_code                                                            
Name: 841, dtype: object
POINT (83.45657524506824 27.60133891391727)
POINT (83.45659541807774 27.60143672316237)
POINT (83.45659543827094 27.60143682106952)
1
841
146
OBJECTID_1                                                                 843
NEW                                                                     TW1256
geometry                       POINT Z (83.45637524692017 27

POINT (83.45175906981004 27.60687257014285)
POINT (83.45172078952972 27.60678029832296)
POINT (83.45175910812864 27.60687266250704)
-1
853
101
OBJECTID_1                                                                 855
NEW                                                                      TW431
geometry                       POINT Z (83.45167989101424 27.60690833455203 0)
road_direction                                                            None
distance2_intersecting_road                                                  0
road_length_d_code                                                            
Name: 854, dtype: object
POINT (83.45176006358207 27.60687496555777)
POINT (83.45172178330175 27.60678269373787)
POINT (83.45176010190067 27.60687505792195)
-1
854
101
OBJECTID_1                                                                 856
NEW                                                                      TW439
geometry                       POINT Z (83.45196542666315

In [ ]:
buildings.to_file("output_building_1.geojson", driver="GeoJSON")

In [ ]:
print(buildings)

In [ ]:
road.to_file("output_road_1.geojson", driver="GeoJSON")